In [2]:
import io
import streamlit as st
import pandas as pd
from openai import OpenAI

In [4]:
client = OpenAI(api_key=st.secrets.store_api_key.OPENAI_API_KEY)

VECTOR_ID_CTC_DESCOMPLICA_GERAL = st.secrets.store_api_key.VECTOR_ID_CTC_DESCOMPLICA_GERAL
ASSISTANT_ID_CTC_DESCOMPLICA_GERAL = st.secrets.store_api_key.ASSISTANT_ID_CTC_DESCOMPLICA_GERAL

print(f"VECTOR_ID_CTC_DESCOMPLICA_GERAL = {VECTOR_ID_CTC_DESCOMPLICA_GERAL}")
print(f"ASSISTANT_ID_CTC_DESCOMPLICA_GERAL = {ASSISTANT_ID_CTC_DESCOMPLICA_GERAL}")

VECTOR_ID_CTC_DESCOMPLICA_GERAL = vs_vxzHJfQ1ltACO7iYMtkUsiSd
ASSISTANT_ID_CTC_DESCOMPLICA_GERAL = asst_DMZBpA4feCFfNkMtOFrNHVNR


In [ ]:

list_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

assistant_ja_existe = False
for assistant in list_assistants.data:
    if "CTC_DESCOMPLICA_GERAL" == assistant.name:
        print(assistant)
        assistant_ja_existe = True
        break

if not assistant_ja_existe:
    assistent = client.beta.assistants.create(
        instructions="Voce é um assistente do departamento de recursos humanos da empresa CTC, você responde as perguntas enviadas somente com base nos dados encontrados no seu vector store, não responda outros assuntos, seja objetivo, caso não encontre a resposta no vector storeresponda 'Não encontrei informações que possa ajudar na sua resposta!'",
        name="CTC_DESCOMPLICA_GERAL",
        tools=[{"type": "file_search"}],
        tool_resources={"file_search": {"vector_store_ids": [VECTOR_ID_CTC_DESCOMPLICA_GERAL]}},
        model="gpt-4o"
    )
    print("Assistente Criado!")
else:
    print("assistant_ja_existe",assistant_ja_existe)


VECTOR_ID_CTC_DESCOMPLICA_GERAL = vs_vxzHJfQ1ltACO7iYMtkUsiSd
ASSISTANT_ID_CTC_DESCOMPLICA_GERAL = asst_DMZBpA4feCFfNkMtOFrNHVNR
Assistant(id='asst_1xQv1BPjNNz7aHIdRFthc199', created_at=1733770548, description=None, instructions="Voce é um assistente do departamento de recursos humanos da empresa CTC, você responde as perguntas enviadas somente com base nos dados encontrados no seu vector store, não responda outros assuntos, seja objetivo, caso não encontre a resposta no vector storeresponda 'Não encontrei informações que possa ajudar na sua resposta!'", metadata={}, model='gpt-4o', name='CTC_DESCOMPLICA_GERAL', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_vxzHJfQ1ltACO7iYMtkUsiSd'])

In [14]:
data = [{ "ID": 1,
            "pergunta": "Qual o dia do pagamento?",
            "resposta": "Até o dia 10 de cada mês, caso dia 10 for um feriado! será no próximo dia útil.",
            "data": "2023-12-08",
            "resposavel" : "RH"
        },{ "ID": 2,
            "pergunta": "Qual o dia do aniversário da empresa CTC ?",
            "resposta": "O aniversário da empresa é 1 de janeiro.",
            "data": "2023-12-08",
            "resposavel" : "Diretoria"
        },]

df = pd.DataFrame( data )
df['data'] = pd.to_datetime(df['data']).dt.date

df

,ID,pergunta,resposta,data,resposavel
0,1,Qual o dia do pagamento?,"Até o dia 10 de cada mês, caso dia 10 for um f...",2023-12-08,RH
1,2,Qual o dia do aniversário da empresa CTC ?,O aniversário da empresa é 1 de janeiro.,2023-12-08,Diretoria


In [5]:
list_vector_store_files = client.beta.vector_stores.files.list(
    vector_store_id=VECTOR_ID_CTC_DESCOMPLICA_GERAL
)
print(f"vector_store_files.data = {list_vector_store_files.data}")

for vector_store_file in list_vector_store_files.data:
    print(f"vector_store_file.id = {vector_store_file.id}")

    #deleted_vector_store_file = client.beta.vector_stores.files.delete(
    #    vector_store_id=VECTOR_ID_CTC_DESCOMPLICA_GERAL,
    #    file_id=vector_store_file.id
    #)
    #print(deleted_vector_store_file)

vector_store_files.data = [VectorStoreFile(id='file-6N2Pxuboeqcmz3kpCCUb9h', created_at=1733782844, last_error=None, object='vector_store.file', status='completed', usage_bytes=1935, vector_store_id='vs_vxzHJfQ1ltACO7iYMtkUsiSd', chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))]
vector_store_file.id = file-6N2Pxuboeqcmz3kpCCUb9h


In [11]:
list_files = client.files.list()

for arquivo in list_files.data:
    print(arquivo.id)

file-6N2Pxuboeqcmz3kpCCUb9h
file-DnptX7pGCNCfGUfjCpXHo7
file-XhBcfPmh1AGh9EvuEoLN84
file-5BCk1DoWTpeyRXK9pf1FmU
file-VNq2M4Rh4Ltwox3zLkHEp2


In [78]:
json_buffer = io.BytesIO()
df.to_json(json_buffer, orient='records')
json_buffer.seek(0)
json_buffer.name = 'CTC_DESCOMPLICA_GERAL.json'

file = client.files.create(file=json_buffer, purpose='assistants')
print(f"file = {file}")

create_vector_store_file = client.beta.vector_stores.files.create(
    vector_store_id=VECTOR_ID_CTC_DESCOMPLICA_GERAL,
    file_id = file.id
)

print(f"vector_store_file = {create_vector_store_file}")

file = FileObject(id='file-BbFkiWbAcJnbhWvSdrveSY', bytes=385, created_at=1733774681, filename='CTC_DESCOMPLICA_GERAL.json', object='file', purpose='assistants', status='processed', status_details=None)
vector_store_file = VectorStoreFile(id='file-BbFkiWbAcJnbhWvSdrveSY', created_at=1733774681, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_vxzHJfQ1ltACO7iYMtkUsiSd', chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))
